# <h1 align = 'center'> Data collection, Transmission and Security part 5</h1> 
#### <center> Marvel VS DC  </center>
#### <center> Achraf BELLA - Ecole Centrale - Casablanca - January 2022 </center>
***

In [89]:
import pandas as pd
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
import os

**Let's use for exemple marvel_data_usa for instance**

In [90]:
marvel_usa = pd.read_pickle('marvel_data_usa_df.pkl')

**we took only the data **

In [91]:
df = marvel_usa[['tweet']] 
del marvel_usa

# Part 1

### This function help you to encrypt the tweets

In [92]:
def encrypt(key, plaintext, associated_data):
    # Generate a random 96-bit IV.
    iv = os.urandom(12)

    # Construct an AES-GCM Cipher object with the given key and a
    # randomly generated IV.
    encryptor = Cipher(
        algorithms.AES(key),
        modes.GCM(iv),
    ).encryptor()

    # associated_data will be authenticated but not encrypted,
    # it must also be passed in on decryption.
    encryptor.authenticate_additional_data(associated_data)

    # Encrypt the plaintext and get the associated ciphertext.
    # GCM does not require padding.
    plaintext = bytes(plaintext, encoding='utf-8')
    ciphertext = encryptor.update(plaintext) + encryptor.finalize()

    return  iv, ciphertext, encryptor.tag 

In [93]:
associated_data = b"authenticated but not encrypted payload" 
key = os.urandom(32)

In [94]:
df['information'] = df.tweet.apply(lambda x : encrypt(key, x, associated_data))

In [95]:
iv = df.information.apply(lambda x : x[0])
encrypted_tweet = df.information.apply(lambda x : x[1])
tags = df.information.apply(lambda x : x[2])

**I get rid of all the dataframe in order to send you only the encrypted_tweet**  

In [96]:
del df

### Here you have the encrypted tweets

In [97]:
encrypted_tweet

0         b'\xad\x8f\n\xa6\x906p\xf8=\xd9C\xdcM\xec*\xab...
1         b'\x1f\x99\t\xe7iYH-E\xe7as\xcc\xebS\xa5YY\xf1...
2         b'3\x9d\xbcP\x8d\xde\x9aw\xe8\x16\xaa"I\x9dr\x...
3         b"\xd6\xac\x14)\x18\x88\x83 732\x17#\x01Bx\x94...
4         b'\xf0`ko\xf3h\xf1a\xefj\xa1s8}\xe3\xa8\xd2\xe...
                                ...                        
152926    b'\xb4\x16p\xdd\xf7\xcfF\x99\x19\xf4\xba\xae\x...
152928    b'\xd8n\x86\xf5\xcbs{|\x17\x16\xe3L\x8b]\xa0\x...
152942    b'\x7f\x08\xe9j\xa1\x890\x99d\xafU\x15\xcc\xb5...
152949    b'\x0f.\xeel\x8c\xbb\xdbd\x07\xe4\x05\xf8[\xd2...
153309    b'^z\xd3=&7\xb2\xb37\xc9\xc2\xffVVO\x0f?G\xf6z...
Name: information, Length: 32391, dtype: object

# Part 2

### This function help you to decrypt the hashcode

In [98]:
def decrypt(key, associated_data, iv, ciphertext, tag):
    # Construct a Cipher object, with the key, iv, and additionally the
    # GCM tag used for authenticating the message.
    decryptor = Cipher(
        algorithms.AES(key),
        modes.GCM(iv, tag),
    ).decryptor()

    # We put associated_data back in or the tag will fail to verify
    # when we finalize the decryptor.
    decryptor.authenticate_additional_data(associated_data)

    # Decryption gets us the authenticated plaintext.
    # If the tag does not match an InvalidTag exception will be raised.
    return decryptor.update(ciphertext) + decryptor.finalize()


**To decrypt a specific tweet you need the key, iv and the tag**

In [101]:
decrypt(key, associated_data, iv[0], encrypted_tweet[0], tags[0])

b'@KnightWing19 Marvel Comics makes too many X-Men books like how DC Comics makes too many Batman books.'

Here I encrypted the 1  tweet  as you can see